In [2]:
!pip install openai
!pip install langchain_openai

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [9]:
import os
from openai import OpenAI

client = OpenAI(
    api_key="KEY",
)


completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are a smart home assistant. Your task is generating Trigger-Action Rules for Home Automation"},
    {"role": "user", "content": "I want to turn off lights when I am not at home"}
  ]
)



print(completion.choices[0].message)

ChatCompletionMessage(content='Sure! Here\'s a Trigger-Action Rule (TAR) for that scenario:\n\n**Trigger**: Your smartphone disconnects from the home Wi-Fi network (indicating that you have left home).\n\n**Action**: Turn off all the lights in your home.\n\nThis can be configured using your home automation system or application. Here is a more detailed rule if you are using a typical smart home platform:\n\n1. **Trigger**: If the smartphone’s presence detection shows "away".\n2. **Action**: \n   - Turn off light 1.\n   - Turn off light 2.\n   - ...\n   - Turn off light n.\n\nAlternatively, you could use geofencing capabilities where the trigger involves your smartphone leaving a specified geographic area around your home instead of just leaving the Wi-Fi network.', role='assistant', function_call=None, tool_calls=None)


In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from time import time

os.environ['OPENAI_API_KEY'] = "sk-proj-OwMaIU01JLzdwYBDaV10T3BlbkFJ07ZTBUuM88fcMiub7OvV"

import subprocess
from openai import OpenAI

def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

# List the devices
devices_output = list_smartthings_devices()

prompt = ChatPromptTemplate.from_template("You are a smart home assistant. The devices are in {devices_output}. Your task is generating Trigger-Action Rules for Home Automation")

output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

chain = (
    prompt 
    | model 
    | output_parser
)

input_data = {
    "devices_output": devices_output
}

result = chain.invoke(input_data)

print(result)


Sure! Here are some trigger-action rules for home automation based on your device list:

### Rule 1: Turn Off the Centralite Outlet When Motion is Detected
**Trigger:** Motion detected by the "Motion Sensor"  
**Action:** Turn off the "Centralite Outlet 1"

**Rule Definition:**
```json
{
    "trigger": {
        "deviceId": "badd4ded-6183-4fb6-928c-179dcd6d1229",
        "capability": "motionSensor",
        "attribute": "motion",
        "value": "active"
    },
    "action": {
        "deviceId": "03f87109-2da4-4376-8372-580372d9ded6",
        "capability": "switch",
        "attribute": "switch",
        "command": "off"
    }
}
```

### Rule 2: Turn On the Aeotec Outlet at a Specific Time
**Trigger:** Time-based (e.g., 7:00 AM every day)  
**Action:** Turn on the "Aeotec Outlet 1"

**Rule Definition:**
```json
{
    "trigger": {
        "type": "time",
        "time": "07:00:00"
    },
    "action": {
        "deviceId": "7b66afce-2f5a-4bc4-b55f-6b949389f86d",
        "capability":